In [ ]:
%load_ext autoreload
%autoreload 2

import coint_trading as ct
import data_io as myio
import pandas as pd
import utils as ut
import model_settings as settings

import warnings
warnings.filterwarnings("ignore")

### User specificaiton

In [ ]:
from itertools import combinations 

G8_no_CHF_list = ['EURUSD Curncy', 'GBPUSD Curncy', 'AUDUSD Curncy', 'NZDUSD Curncy', 
                  'JPYUSD Curncy', 'CADUSD Curncy', 'SGDUSD Curncy', 'CNHUSD Curncy']

df_fx_spot, df_fx_ivol = myio.read_all_fx_data('../data/bloomberg fx data sheet.xlsx')

TRADE_DATE_list = ut.generate_Mon_to_Fri_between_2_dates('2020-02-25', '2020-06-26')

FROM_list = ['2015-07-23', '2015-10-23',
             '2016-01-23', '2016-04-23'
            ]
             
output_xls = '../multi_dates_coint_search_for_back_testing_on_' + df_fx_spot.index[-1].strftime('%Y-%m-%d') + '.xlsx' 

print(len(TRADE_DATE_list))

### Performing the Johansen Test on all basket combinations

In [ ]:
SHOW_BASKET_WITH_SD_ABOVE = 2.0
SHOW_BASKET_WITH_SHARPE_ABOVE = 0.5
SHOW_BASKET_WITH_HALF_LIFE_IN_DAYS_BELOW = 80

output_df = pd.DataFrame()

tic = ut.time_now()

for TRADE_DATE in TRADE_DATE_list:
    print(TRADE_DATE)    
    
    res_df = ct.johansen_compute_all_baskets(FROM_list, TRADE_DATE, G8_no_CHF_list, df_fx_spot)   
    
    df_filtered = ct.filter_cointegration_basket_using_trace_10(res_df)    
    df_filtered = ct.compute_johansen_test_diag_for_all_coint_baskets(df_filtered, df_fx_spot)
    df_filtered = ct.filter_cointegration_basket_using_SD_threshold(df_filtered, SHOW_BASKET_WITH_SD_ABOVE)  
    df_filtered = ct.filter_cointegration_basket_using_1m_1SD_in_USD(df_filtered)
    df_filtered = ct.filter_cointegration_basket_using_Sharpe_Ratio(df_filtered, SHOW_BASKET_WITH_SHARPE_ABOVE)
    df_filtered = ct.filter_cointegration_basket_using_half_life_in_days(df_filtered, SHOW_BASKET_WITH_HALF_LIFE_IN_DAYS_BELOW)

    df_filtered = ct.compute_johansen_stability_diagnostics(df_filtered, df_fx_spot, settings.DataFreq.DAILY)
    df_filtered = ct.compute_historical_min_max_SD_diagnostics(df_filtered, df_fx_spot)
    
    output_df = pd.concat([output_df, df_filtered])
    
output_df.to_excel(output_xls)

toc = ut.time_now()
print(toc - tic)